In [ ]:
import pandas as pd
import numpy as np
import pickle

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from sklearn.metrics import (mean_squared_error, mean_absolute_error, mean_absolute_percentage_error)

# Arima
from pmdarima import auto_arima
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from datetime import datetime, timedelta

# Neural Prophet
from neuralprophet import NeuralProphet
from neuralprophet import set_random_seed 
set_random_seed(0)  # control the random initialization of weights

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

## DATA

In [ ]:
# Lectura de datos
df = pd.read_parquet("forecast_data.parquet")
df.head()

Vamos a evaluar diferentes técnicas de forecasting para poder predecir los futuros retrasos de la manera más precisa posible. Para ello seleccionamos los datos correspondientes a un único aeropuerto, que será con el cual realizaremos las pruebas de modelado

In [ ]:
airports = list(df['ORIGIN_AIRPORT'].unique())
airports

In [ ]:
# Seleccionamos el primer aeropuerto de la lista
data = df[df['ORIGIN_AIRPORT'] == airports[0]]
data = data.loc[:,['DATE', 'DELAYED_FLIGHTS']] 

# Cambiamos el nombre a las columnas para que vaya a corde con el modelo
data.columns = ['ds','y']
data['ds'] = pd.to_datetime(data['ds'],format = "%m/%d/%Y")
data.head()

# Forecast

Variable a predecir: *number of delays per airport and day*

## Option I. Arima

Modelo ARIMA para la predicción de series de tiempo
ARIMA significa modelo de promedio móvil integrado autorregresivo y se especifica mediante tres parámetros de orden: (p, d, q).

AR( p ) Autoregresión : un modelo de regresión que utiliza la relación dependiente entre una observación actual y las observaciones durante un período anterior Un componente auto regresivo( AR(p) ) se refiere al uso de valores pasados ​​en la ecuación de regresión para la serie de tiempo.

I( d ) Integración : utiliza la diferenciación de observaciones(restando una observación de la observación en el paso de tiempo anterior) para hacer estacionaria la serie de tiempo. La diferenciación implica la resta de los valores actuales de una serie con sus valores anteriores d número de veces.<p>
Media móvil MA( q ) : un modelo que utiliza la dependencia entre una observación y un error residual de un modelo de media móvil aplicado a observaciones retrasadas. Un componente de media móvil representa el error del modelo como una combinación de términos de error anteriores. El orden q representa el número de términos que se incluirán en el modelo.

In [ ]:
stepwise_fit = auto_arima(data['y'], start_p = 1, start_q = 1,max_p = 3, max_q = 3, m = 12,start_P = 0, 
                          seasonal = True, d = None, D = 1, trace = True,error_action ='ignore',  
                          suppress_warnings = True,stepwise = True)          
  
stepwise_fit.summary()

In [ ]:
# Ajustamos el modelo al conjunto de datos
train = data.iloc[:len(data)-31]
test = data.iloc[len(data)-31:]

arima_model = SARIMAX(train['y'],order = (1, 0, 0),seasonal_order =(2, 1, 0, 12))
  
result = arima_model.fit()
result.summary()

In [ ]:
# Predictions
start = len(train)
end = len(train)+len(test)-1
  
predictions = result.predict(start, end,typ = 'levels').rename("Predictions")
data

In [ ]:
# Predictions
start = len(train)
end = len(train)+len(test)-1
  
predictions = result.predict(start, end,typ = 'levels').rename("Predictions")
test['predictions'] = predictions

# Prediction vs Actual values representation

fig = go.Figure()

fig.add_trace(go.Scatter(x=test['ds'], y=test['y'], name = "actual", line_color = px.colors.qualitative.Vivid[5]))
fig.add_trace(go.Scatter(x=test['ds'], y=test['predictions'], name = "predictions",line_color = px.colors.qualitative.Vivid[3]))

fig.update_layout(
    title="ARIMA Model. December predictions vs. Actual values",
    xaxis_title="Dates",
    yaxis_title="Number of delays",
    legend_title="Leyend",
    template="plotly_dark",
    #color_discrete_sequence=px.colors.qualitative.Vivid,
    hovermode="x unified",
       
)

fig.show()


In [ ]:
# Check the MSE value
performance_arima_MSE = mean_squared_error(test['y'],test['predictions'])
print(f'The MSE for the baseline model is {performance_arima_MSE}')

# Check the MAE value
performance_arima_MAE = mean_absolute_error(test['y'],test['predictions'])
print(f'The MAE for the baseline model is {performance_arima_MAE}')

# Check the MAPE value
performance_arima_MAPE = mean_absolute_percentage_error(test['y'],test['predictions'])
print(f'The MAPE for the baseline model is {performance_arima_MAPE}')

In [ ]:
# Predicciones futuras. Vamos a estimar los retrasos futuros correspondientes a enero 2016
model = SARIMAX(data['y'], order = (1, 0, 0),seasonal_order =(2, 1, 0, 12))
arima_result = model.fit()
  
forecast = arima_result.predict(start = len(data),end = (len(data)-1) + 31,typ = 'levels').rename('Forecast')

jan_2016 = pd.DataFrame()
jan_2016['ds'] = [datetime(2016,1,1) + timedelta(days=d) for d in range((datetime(2016,1,31) - datetime(2016,1,1)).days + 1)] 
jan_2016['predictions'] = list(forecast)

In [ ]:
# Forecast representation
fig = go.Figure()

fig.add_trace(go.Scatter(x=data['ds'], y=data['y'], name = "Year 2015", line_color = px.colors.qualitative.Vivid[5]))
fig.add_trace(go.Scatter(x=jan_2016['ds'], y=jan_2016['predictions'], name = "Jan 2016 forecast",line_color = px.colors.qualitative.Vivid[3]))

fig.update_layout(
    title="ARIMA Model. January 2016 predictions",
    xaxis_title="Dates",
    yaxis_title="Number of delays",
    legend_title="Leyend",
    template="plotly_dark",
    hovermode="x unified",
       
)

fig.show()

In [ ]:
# Por último vamos a representar los datos de enero de 2015 con los de 2016.
# Para poder comparar ambos resultados cambiamos el año de nuestros datos del 2015 al 2016 y representamos
jan_2015 = data.iloc[0:31,:]
jan_2015['ds'] = jan_2016['ds']

fig = go.Figure()

fig.add_trace(go.Scatter(x=jan_2015['ds'], y=jan_2015['y'], name = "Jan 2015", line_color = px.colors.qualitative.Vivid[5]))
fig.add_trace(go.Scatter(x=jan_2016['ds'], y=jan_2016['predictions'], name = "Jan 2016 forecast",line_color = px.colors.qualitative.Vivid[3]))

fig.update_layout(
    title="ARIMA Model. January 2016 predictions",
    xaxis_title="Dates",
    yaxis_title="Number of delays",
    legend_title="Leyend",
    template="plotly_dark",
    hovermode="x unified",
       
)

fig.show()


In [ ]:
# # Check the MSE value
# performance_arima_jan_MSE = mean_squared_error(jan_2015['y'],jan_2016['predictions'])
# print(f'The MSE for the baseline model is {performance_arima_jan_MSE}')

# # Check the MAE value
# performance_arima_jan_MAE = mean_absolute_error(jan_2015['y'],jan_2016['predictions'])
# print(f'The MAE for the baseline model is {performance_arima_jan_MAE}')

# # Check the MAPE value
# performance_arima__jan_MAPE = mean_absolute_percentage_error(jan_2015['y'],jan_2016['predictions'])
# print(f'The MAPE for the baseline model is {performance_arima__jan_MAPE}')

## Neural Prophet

Model variables explained in: https://neuralprophet.com/code/forecaster.html

Quick start guide: https://neuralprophet.com/quickstart.html

###  Data partition

In [ ]:
# Train test split
# Vamos a predecir el mes de diciembre
train_end_date = '2015-11-30'

train = data[data['ds'] <= train_end_date]
test  = data[data['ds'] >  train_end_date]

# Check the shape of the dataset
print(train.shape)
print(test.shape)

###  Base Model

In [ ]:
# Build the model
model = NeuralProphet()
model.fit(train)

In [ ]:
# Create the time range for the forecast
future_baseline = model.make_future_dataframe(train,periods = 31)

# Make prediction
forecast_baseline = model.predict(future_baseline)

The forecast dataframe does not include the actual values so we need to merge the forecast dataframe with the test dataframe to compare the actual values with the predicted values

In [ ]:
# Merge actual and predicted values
performance_baseline = pd.merge(test,forecast_baseline[['ds','residual1','yhat1','trend','season_weekly']][-31:],on='ds')

# Check the MSE value
performance_baseline_MSE = mean_squared_error(performance_baseline['y'],performance_baseline['yhat1'])
print(f'The MSE for the baseline model is {performance_baseline_MSE}')

# Check the MAE value
performance_baseline_MAE = mean_absolute_error(performance_baseline['y'],performance_baseline['yhat1'])
print(f'The MAE for the baseline model is {performance_baseline_MAE}')

# Check the MAPE value
performance_baseline_MAPE = mean_absolute_percentage_error(performance_baseline['y'],performance_baseline['yhat1'])
print(f'The MAPE for the baseline model is {performance_baseline_MAPE}')

In [ ]:
# Prediction vs Actual values representation
fig = go.Figure()

fig.add_trace(go.Scatter(x=performance_baseline['ds'], y=performance_baseline['y'], name = "actual", line_color = px.colors.qualitative.Vivid[5]))
fig.add_trace(go.Scatter(x=performance_baseline['ds'], y=performance_baseline['yhat1'], name = "predictions",line_color = px.colors.qualitative.Vivid[3]))

fig.update_layout(
    title="Baseline Model. December predictions vs. Actual values",
    xaxis_title="Dates",
    yaxis_title="Number of delays",
    legend_title="Leyend",
    template="plotly_dark",
    #color_discrete_sequence=px.colors.qualitative.Vivid,
    hovermode="x unified",
       
)

fig.show()

### Seasonality Study

In [ ]:
# Representación de los datos
fig = px.line(data, x="ds", y="y", labels={'ds':'Date', 'y':'Number of delays'},
              title = "Evolution of delayed flights per day",color_discrete_sequence=px.colors.qualitative.Vivid, template="plotly_dark")

fig.show()

#### Weekly seasonality

In [ ]:
aux = pd.DataFrame()
aux["DELAYED_FLIGHTS"] = data.groupby(data['ds'].dt.day_name())["y"].sum()
#data["Vuelos Retrasados"] = data[data["ARRIVAL_DELAY"]>0].groupby(data['DATE'].dt.day_name())["FLIGHT_NUMBER"].count()
aux = aux.reindex(index = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
aux = aux.reset_index(level=0, drop=False)
aux

In [ ]:
# Representación de los datos
fig = px.line(aux, x="ds", y="DELAYED_FLIGHTS", labels={'ds':'Date', 'DELAYED_FLIGHTS':'Number of delays'},
              title = "Delayed flights per Week day",color_discrete_sequence=px.colors.qualitative.Vivid, template="plotly_dark")

#fig.update_traces({"line":{"color":"steelblue", 'dash':'dash'}})

fig.show()

In [ ]:
aux = pd.DataFrame()
aux["DELAYED_FLIGHTS"] = data.groupby(data['ds'].dt.to_period('M'))["y"].sum()
aux = aux.rename(columns={"ds":"Month"})
aux['MONTH'] =['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
aux = aux.reset_index(level=0, drop=False)
aux

#### Monthly seasonality

In [ ]:
fig = px.bar(aux, x='MONTH', y='DELAYED_FLIGHTS',
            title = "Delayed flights per Month",color_discrete_sequence=px.colors.qualitative.Vivid, template="plotly_dark")

#fig.add_trace(px.line(aux, x='MONTH', y='DELAYED_FLIGHTS'))

fig.show()

### Model with Seasonality

In [ ]:
# No añadimos yearly porque solo tenemos datos de un año ni tampoco daily porque nuestro dataset está organizado por días
model_season = NeuralProphet(weekly_seasonality = True)

# Fit the model on the training dataset
model_season.fit(train)

In [ ]:
# Create the time range for the forecast
future_season = model_season.make_future_dataframe(train,periods = 31)

# Make prediction
forecast_season = model_season.predict(future_season)

In [ ]:
# Merge actual and predicted values
performance_season = pd.merge(test,forecast_season[['ds','residual1','yhat1','trend','season_weekly']][-31:],on='ds')

# Check the MSE value
performance_season_MSE = mean_squared_error(performance_season['y'],performance_season['yhat1'])
print(f'The MSE for the baseline model is {performance_season_MSE}')

# Check the MAE value
performance_season_MAE = mean_absolute_error(performance_season['y'],performance_season['yhat1'])
print(f'The MAE for the baseline model is {performance_season_MAE}')

# Check the MAPE value
performance_season_MAPE = mean_absolute_percentage_error(performance_season['y'],performance_season['yhat1'])
print(f'The MAPE for the baseline model is {performance_season_MAPE}')

In [ ]:
# Prediction vs Actual values representation
fig = go.Figure()

fig.add_trace(go.Scatter(x=performance_season['ds'], y=performance_season['y'], name = "actual", line_color = px.colors.qualitative.Vivid[5]))
fig.add_trace(go.Scatter(x=performance_season['ds'], y=performance_season['yhat1'], name = "predictions",line_color = px.colors.qualitative.Vivid[3]))

fig.update_layout(
    title="Model with seasonality. December predictions vs. Actual values",
    xaxis_title="Dates",
    yaxis_title="Number of delays",
    legend_title="Leyend",
    template="plotly_dark",
    hovermode="x unified",
       
)

fig.show()

### Tunned Model. Seasonality & holidays

In [ ]:
model_tunned = NeuralProphet(weekly_seasonality = True)
model_tunned.add_country_holidays(country_name='US')
model_tunned.fit(train)

In [ ]:
# Create the time range for the forecast
future_tunned = model_tunned.make_future_dataframe(train,periods = 31)

# Make prediction
forecast_tunned = model_tunned.predict(future_tunned)

In [ ]:
# Merge actual and predicted values
performance_tunned = pd.merge(test,forecast_tunned[['ds','residual1','yhat1','trend','season_weekly']][-31:],on='ds')

# Check the MSE value
performance_tunned_MSE = mean_squared_error(performance_tunned['y'],performance_tunned['yhat1'])
print(f'The MSE for the baseline model is {performance_tunned_MSE}')

# Check the MAE value
performance_tunned_MAE = mean_absolute_error(performance_tunned['y'],performance_tunned['yhat1'])
print(f'The MAE for the baseline model is {performance_tunned_MAE}')

# Check the MAPE value
performance_tunned_MAPE = mean_absolute_percentage_error(performance_tunned['y'],performance_tunned['yhat1'])
print(f'The MAPE for the baseline model is {performance_tunned_MAPE}')

In [ ]:
# Prediction vs Actual values representation
fig = go.Figure()

fig.add_trace(go.Scatter(x=performance_tunned['ds'], y=performance_tunned['y'], name = "actual", line_color = px.colors.qualitative.Vivid[5]))
fig.add_trace(go.Scatter(x=performance_tunned['ds'], y=performance_tunned['yhat1'], name = "predictions",line_color = px.colors.qualitative.Vivid[3]))

fig.update_layout(
    title="Model Tunned. December predictions vs. Actual values",
    xaxis_title="Dates",
    yaxis_title="Number of delays",
    legend_title="Leyend",
    template="plotly_dark",
    hovermode="x unified",     
)

fig.show()

### Tunned II

In [ ]:
model_tunned_II = NeuralProphet(   
    growth="off",              # no apparent trend
    yearly_seasonality=False,  # not enough data
    weekly_seasonality=True,
    daily_seasonality=False,   # not hourly data
    seasonality_reg=0,         # modulates the strength of the seasinality model --> fit larger seasonality fluctuations
    loss_func="Huber",         # Loss function that is less sensitive to outliers in data than the squared error loss.
    normalize="auto",          # Type of normalization ('minmax', 'standardize', 'soft', 'off')
)
model_tunned_II.add_country_holidays(country_name='US')
model_tunned_II.fit(train)

In [ ]:
# Create the time range for the forecast
future_tunned_II = model_tunned_II.make_future_dataframe(train,periods = 31)

# Make prediction
forecast_tunned_II = model_tunned_II.predict(future_tunned_II)

In [ ]:
# Merge actual and predicted values
performance_tunned_II = pd.merge(test,forecast_tunned_II[['ds','residual1','yhat1','trend','season_weekly']][-31:],on='ds')

# Check the MSE value
performance_tunned_MSE_II = mean_squared_error(performance_tunned_II['y'],performance_tunned_II['yhat1'])
print(f'The MSE for the baseline model is {performance_tunned_MSE_II}')

# Check the MAE value
performance_tunned_MAE_II = mean_absolute_error(performance_tunned_II['y'],performance_tunned_II['yhat1'])
print(f'The MAE for the baseline model is {performance_tunned_MAE_II}')

# Check the MAPE value
performance_tunned_MAPE_II = mean_absolute_percentage_error(performance_tunned_II['y'],performance_tunned_II['yhat1'])
print(f'The MAPE for the baseline model is {performance_tunned_MAPE_II}')

In [ ]:
# Prediction vs Actual values representation
fig = go.Figure()

fig.add_trace(go.Scatter(x=performance_tunned_II['ds'], y=performance_tunned_II['y'], name = "actual", line_color = px.colors.qualitative.Vivid[5]))
fig.add_trace(go.Scatter(x=performance_tunned_II['ds'], y=performance_tunned_II['yhat1'], name = "predictions",line_color = px.colors.qualitative.Vivid[3]))

fig.update_layout(
    title="Model Tunned II. December predictions vs. Actual values",
    xaxis_title="Dates",
    yaxis_title="Number of delays",
    legend_title="Leyend",
    template="plotly_dark",
    hovermode="x unified",     
)

fig.show()

### Comparativa modelos

In [ ]:
# Representación de los dos modelos
fig = go.Figure()
fig.add_trace(go.Scatter(x=performance_baseline['ds'], y=performance_baseline['y'], name = "actual", line_color = px.colors.qualitative.Vivid[1]))
fig.add_trace(go.Scatter(x=performance_baseline['ds'], y=performance_baseline['yhat1'], name = "baseline", line_color = px.colors.qualitative.Vivid[5]))
fig.add_trace(go.Scatter(x=performance_season['ds'], y=performance_season['yhat1'], name = "season",line_color = px.colors.qualitative.Vivid[3],mode='markers'))
fig.add_trace(go.Scatter(x=performance_season['ds'], y=performance_tunned['yhat1'], name = "tunned",line_color = px.colors.qualitative.Vivid[2],mode='lines+markers'))
fig.add_trace(go.Scatter(x=performance_season['ds'], y=performance_tunned_II['yhat1'], name = "tunned II",line_color = px.colors.qualitative.Vivid[4]))

fig.update_layout(
    title="Models Comparison",
    xaxis_title="Dates",
    yaxis_title="Number of delays",
    legend_title="Leyend",
    template="plotly_dark",
    hovermode="x unified",       
)

fig.show()

## Chosen model: ARIMA